This Jupyter Noteboox shows how to perform spatial univariate pre-whitening of GLM coefficients using the mean residual sums of squares (ResMS) output by SPM after GLM estimation. 

In [1]:
# relevant imports
import numpy as np
import rsatoolbox as rsa
from scipy.linalg import sqrtm
import nibabel as nb

np.random.seed(7)


GLM parameters (betas) stored in .nii image files after SPM estimation can be loaded using nibabel.load() and then extracted as an i-by-j-by-k 3D tensor with the .get_fdata() method.

Alternatively, you can load betas from a specific ROI using world coordinates with nitools.sample_image(). This may help you save space of disk. In this case, betas from the desired locations are stored as a vector.

Here, we will simulate a vector of betas similar to the output of nitools.sample_image().

In [7]:
betas = np.random.random(size=(1000,))

Univariate pre-whitening involves normalizing the activity estimated in each voxel by the standard deviation of its residuals. SPM stores the residual mean squares in the ResMS.nii file in the same directory as the beta_XXXX.nii files. You can load them using nibabel.load('path/to/glmDir/ResMS.nii').get_fdata() or nitools.sample_image(). Here, we will simulate a vector of mean squares as done for the betas.

In [8]:
# load residual mean square
ResMS = np.random.random(size=(1000,))

To pre-whitening our betas, we now need to divide our betas by the square root of the residual mean squares.

In [ ]:
betas_prewhitened = betas / np.sqrt(resMS))

In [16]:
# create a dataset object
nCond = betas_prewhitened.shape[0]
nVox = betas_prewhitened.shape[1]
# now create a dataset object
des = {'session': 1, 'subj': 1}
obs_des = {'conds': np.array(['cond_%02d' % x for x in np.arange(nCond)])}
chn_des = {'voxels': np.array(['voxel_' + str(x) for x in np.arange(nVox)])}
dataset = rsa.data.Dataset(measurements=betas_prewhitened,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)

In [17]:
# calculate euclidean distance between conditions
dist = rsa.rdm.calc_rdm(dataset, method='euclidean', descriptor='conds')
print(dist.get_matrices())

[[[    0.         19372.9029699  60465.35760982 11656.86534238
   17456.97377556]
  [19372.9029699      0.         14103.29222431  6459.20833199
    8810.56894998]
  [60465.35760982 14103.29222431     0.         22045.50967413
   18823.00223702]
  [11656.86534238  6459.20833199 22045.50967413     0.
     913.96644474]
  [17456.97377556  8810.56894998 18823.00223702   913.96644474
       0.        ]]]


In [18]:
# diagonalize ResMS
dataset = rsa.data.Dataset(measurements=betas,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)
dist = rsa.rdm.calc_rdm(dataset, method='mahalanobis', descriptor='conds', noise=np.linalg.inv(Cov))
print(dist.get_matrices())

[[[    0.         19372.9029699  60465.35760982 11656.86534238
   17456.97377556]
  [19372.9029699      0.         14103.29222431  6459.20833199
    8810.56894998]
  [60465.35760982 14103.29222431     0.         22045.50967413
   18823.00223702]
  [11656.86534238  6459.20833199 22045.50967413     0.
     913.96644474]
  [17456.97377556  8810.56894998 18823.00223702   913.96644474
       0.        ]]]
